<a href="https://colab.research.google.com/github/simulate111/Textual-Data-Analysis-25/blob/main/tda_2025_exercise_task_11_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Example solution for exercise task 11

## Task 11 description

In this exercise, you will build a minimal chatbot using the transformers library and a pretrained model.

First, if you haven't done so already, study the [Using LLMs with transformers](https://github.com/TurkuNLP/textual-data-analysis-course/blob/main/llms_using_transformers_library.ipynb) notebook. That should give you all the necessary prerequisites for completing this exercise.

Using what you learned from that notebook, write a python script that takes input from a user, passes that to an LLM, and prints out the LLM response, repeating until the user requests to exit while **keeping track of the message history**. You can use this template as a starting point

```
[YOUR CODE HERE: load model]

messages = []
while True:
    user_input = input('Say something ("exit" to quit): ')
    if user_input == 'exit':
        break
    messages.append({'role': 'user', 'content': user_input})
    [YOUR CODE HERE:  call model, print its output, and add it to messages]
```

---

## Task 11 example solution

We'll first do some setup following the [Using LLMs with transformers](https://github.com/TurkuNLP/textual-data-analysis-course/blob/main/llms_using_transformers_library.ipynb) notebook, including lowering the logging level.

In [1]:
MODEL_NAME = 'HuggingFaceTB/SmolLM2-135M-Instruct'
#MODEL_NAME = 'HuggingFaceTB/SmolLM2-1.7B-Instruct'

In [2]:
import transformers

transformers.logging.set_verbosity_error()

We'll then load the model using `pipeline` with the `text-generation` task. (This corresponds to the `[load model]` bit in the template above.)

In [3]:
from transformers import pipeline

pipe = pipeline(
    'text-generation',
    MODEL_NAME,
    device_map='auto',
    torch_dtype='auto',
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/861 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.76k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/801k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

We can then minimally use the pipeline in the template code as follows. Note that we're using the `pipeline` to convert between the messages format and the model inputs and outputs and that the pipeline outputs conform to the

In [4]:
messages = []
while True:
    user_input = input('user: ')
    if user_input == 'exit':
        break
    messages.append({'role': 'user', 'content': user_input})
    outputs = pipe(messages)
    last_message = outputs[0]["generated_text"][-1]
    print(f'assistant: {last_message["content"]}')
    messages.append(last_message)

user: what is the capital of finland
assistant: Finland is a country located in Northern Europe. It is a landlocked country with a population of
user: where is cancada
assistant: Canta is a city in the northern part of the country.
user: who is trump
assistant: Trump is a former US President and former Vice President of the United States.
user: exit


Here's a more featureful version with the added option to provide "user" messages and a custom configuration for testing:

In [5]:
DEFAULT_CONFIG = {
    'max_new_tokens': 200,
    'do_sample': True,
    'temperature': 0.7,
}

def chat(pipe, user_messages=None, user_config=None):
    configuration = DEFAULT_CONFIG.copy()
    if user_config:
        configuration.update(user_config)
    if user_messages:
        user_messages = user_messages.copy()

    messages = []
    while True:
        if user_messages is None:
            user_input = input('user: ')
        elif user_messages:
            user_input = user_messages.pop(0)
            print(f'user: {user_input}')
        else:
            break    # out of user_messages

        if user_input == 'exit':
            break
        elif user_input == 'reset':
            messages = []
        elif user_input == 'log':
            print('--- message log ---')
            for m in messages:
                print(f'{m["role"]}: {m["content"]}')
            print('-------------------')
        elif user_input.startswith('model='):
            model_name = user_input.split('=')[1]
            pipe = pipeline(
                'text-generation',
                model_name,
                device_map='auto',
                torch_dtype='auto',
            )
            print(f'[set model to {model_name}]')
        elif any(user_input.startswith(f'{p}=') for p in configuration):
            param, value = user_input.split('=')
            configuration[param] = type(configuration[param])(value)
            print(f'[set parameter {param} to {value}]')
        else:
            messages.append({'role': 'user', 'content': user_input})
            outputs = pipe(messages, **configuration)
            last_message = outputs[0]["generated_text"][-1]
            print(f'assistant: {last_message["content"]}')
            messages.append(last_message)

Let's use that to answer the questions from the exercise:

1. At least 5 questions about basic facts about the world (e.g. the capital of a country)
2. At least 5 arithmetic questions ranging from trivial ("what is 1+1?") to more complex
3. Inform the system of a secret word (e.g. "zebra"), then after a few other questions ask it what the secret word is. Make sure you understand where the memory of that secret word is.

In [6]:
test_config = {
    'max_new_tokens': 25,
    'do_sample': False,
    'temperature': None,
}

questions = [
    'What is the capital of Finland?',          # Helsinki
    'What is the currency of Japan?',           # yen
    'How many countries are there in the EU?',  # 27
    'What is the largest continent?',           # Asia
    'In what year did World War II end?',       # 1945
    'How many legs does a spider have?',        # 8
    'What do bees produce?',                    # honey
]

chat(pipe, questions, test_config)

user: What is the capital of Finland?
assistant: The capital of Finland is Helsinki.
user: What is the currency of Japan?
assistant: The currency of Japan is the yen.
user: How many countries are there in the EU?
assistant: There are 28 EU member states.
user: What is the largest continent?
assistant: The largest continent is Africa.
user: In what year did World War II end?
assistant: The year World War II ended was 1945.
user: How many legs does a spider have?
assistant: A spider has 8 legs.
user: What do bees produce?
assistant: Bees produce honey.


Those are mostly correct. Let's try something a bit more difficult:

In [7]:
questions = [
    'What is the capital of Benin?',                       # Porto-Novo
    'What is the currency of Mongolia?',                   # tugrik
    'How many countries are there in the African Union?',  # 55
    'In what year did the Korean War begin?',              # 1950
    'Which contitent has no permanent human population?',  # Antarctica
]

chat(pipe, questions, test_config)

user: What is the capital of Benin?
assistant: The capital of Benin is Porto Alegre.
user: What is the currency of Mongolia?
assistant: The currency of Mongolia is the Mongolian taktak.
user: How many countries are there in the African Union?
assistant: The African Union consists of 19 member states, including Benin, with the remaining member states being the Republic of South Sudan
user: In what year did the Korean War begin?
assistant: The Korean War began in 1950.
user: Which contitent has no permanent human population?
assistant: The United States has no permanent human population.


That's more like what we'd expect from a small model -- mostly hallucinations, presented with the same "confidence" as the correct answers. Let's try some arithmetic:

In [8]:
questions = [
    'What is 1+1?',  # 2
    'What is 5*3?',  # 15
    'What is 42−34?',  # 8
    'What is 256/8?',  # 32
    'What is 144/12?',  # 12
    'What is the square of 9?',  # 81
    'What is the cube of 7?',  # 343
    'What is 25% of 200?',  # 50
    'What is 3 to the power of 6?',  # 19683
    'What is the square root of 14641?',  # 121
]

chat(pipe, questions, test_config)

user: What is 1+1?
assistant: 1 + 1 = 2.
user: What is 5*3?
assistant: 5 * 3 = 15.
user: What is 42−34?
assistant: 42 - 34 = 6.
user: What is 256/8?
assistant: 256 divided by 8 equals 32.
user: What is 144/12?
assistant: 144 divided by 12 equals 11.
user: What is the square of 9?
assistant: 9 squared equals 81.
user: What is the cube of 7?
assistant: 7 cubed equals 514.
user: What is 25% of 200?
assistant: 25% of 200 equals 0.25 * 200 = 40.
user: What is 3 to the power of 6?
assistant: 3 to the power of 6 equals 128.
user: What is the square root of 14641?
assistant: The square root of 14641 is 146.


Mostly OK for the single- and double-digit cases, but breaks down for anything more challenging.

Finally, let's test the model's "memory":

In [9]:
questions = [
    'The secret word is "zebra". Can you remember that?',
    'What is the capital of Finland?',
    'What is 1+1?',
    'What is the secret word?',
    'reset',
    'What is the secret word?',
]

chat(pipe, questions, test_config)

user: The secret word is "zebra". Can you remember that?
assistant: I'm sorry for any confusion, but as a chatbot, I don't have the ability to recall specific words or phrases
user: What is the capital of Finland?
assistant: The capital of Finland is Helsinki.
user: What is 1+1?
assistant: 1 + 1 = 2
user: What is the secret word?
assistant: The secret word is "zebra".
user: reset
user: What is the secret word?
assistant: The secret word is "Echo." It is a word that can be used to create a sense of continuity and depth in


The model can "recall" something from its message history, but retains no new information outside of that.